In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
from copy import deepcopy

import torch
from torch import nn

from utils import train_step, test_step
from utils import BaselineModel
from utils import get_cifar10_dataset, make_dataloader
from utils import check_install_module
check_install_module("plotly")
check_install_module("jupyterplot")

from plotly import express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots

from jupyterplot import ProgressPlot

# Learning rate
![](https://mblogthumb-phinf.pstatic.net/MjAxOTA0MTlfMTg2/MDAxNTU1NjM0NTM2NjQw.ReuAZtSQXIgyGA98iiI_222dHBOMcjMRrew68NU3USUg.NLr7L3nuWQvE4WHGIUoELeQo2nnhghrObFAJVVB-MDQg.PNG.ollehw/image.png?type=w800)

- 여러가지 학습률을 모델이 적용해 훈련이 어떻게 되는지 확인해보기

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_epochs = 5
batch_size = 64
learning_rates = [1., 0.1, 0.01, 0.001]
momentum = 0.9
phases = ['train', 'test']

In [ ]:
cifar10_dataset = get_cifar10_dataset()
loader = make_dataloader(cifar10_dataset, batch_size)

In [ ]:
base_cnn = BaselineModel().to(device)

nets = [BaselineModel().to(device) for _ in learning_rates]
_ = [net.load_state_dict(deepcopy(base_cnn.state_dict())) for net in nets]

optimizers = [
    torch.optim.SGD(net.parameters(), lr, 0.9)
    for net, lr in zip(nets, learning_rates)
]
criterion = nn.CrossEntropyLoss()

pp = ProgressPlot(
    plot_names=phases,
    line_names=list(map(str, learning_rates)),
    x_lim=[0, num_epochs*len(loader['train'])],
    x_label='Iteration',
    y_lim=[[0, 3], [0, 100]]
)
logs = [dict(loss=[], accuracy=[]) for _ in learning_rates]

accs = [0] * len(learning_rates)
for epoch in range(num_epochs):
    for iteration, (inputs, target) in enumerate(loader['train']):
        losses = [
            train_step(net, inputs, target, optimizer, criterion, device)
            for net, optimizer in zip(nets, optimizers)
        ]
        pp.update([losses, accs])
        [log['loss'].append(loss) for log, loss in zip(logs, losses)]
    
    corrects = [0] * len(learning_rates)
    for inputs, target in loader['test']:
        outputs = [
            test_step(net, inputs, target, device=device)[0]
            for net in nets
        ]
        corrects = [
            (correct + (output.argmax(1).cpu() == target).sum()).item()
            for correct, output in zip(corrects, outputs)
        ]
        
    accs = [
        correct / len(cifar10_dataset['test']) * 100
        for correct in corrects
    ]
    [log['accuracy'].append(acc) for log, acc in zip(logs, accs)]
    
    print(f'Epoch: {epoch+1} accuracy ', end='')
    for lr, acc in zip(learning_rates, accs):
        print(f'{lr}: {acc:.2f}', end=' ')
    print()
pp.finalize()

In [ ]:
loss_df = {lr: log['loss'] for lr, log in zip(learning_rates, logs)}
acc_df = {lr: log['accuracy'] for lr, log in zip(learning_rates, logs)}

iteration = list(range(len(loss_df[learning_rates[0]])))
ext_epoch = [iep+len(loader['train']) for iep in range(0, len(iteration), len(loader['train']))]

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_traces(
    [go.Scatter(x=iteration, y=loss_df[lr], name=lr) for lr in learning_rates],
    secondary_ys=[False for _ in learning_rates]
)

fig.add_traces(
    [go.Scatter(x=ext_epoch, y=acc_df[lr], name=lr) for lr in learning_rates],
    secondary_ys=[True for _ in learning_rates]
)

fig.update_xaxes(title_text='Iteration')
fig.update_yaxes(title_text='Loss', secondary_y=False, range=[0, 3])
fig.update_yaxes(title_text='Accuracy', secondary_y=True, range=[0,100])

fig.show()